# Logistic Regression for Sentiment Analysis

Adapted from http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/outofcore_modelpersistence.ipynb

<br>
<br>

## The IMDb Movie Review Dataset

In this section, we will train a simple logistic regression model to classify movie reviews from the 50k IMDb review dataset that has been collected by Maas et. al.

> AL Maas, RE Daly, PT Pham, D Huang, AY Ng, and C Potts. Learning word vectors for sentiment analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Lin- guistics: Human Language Technologies, pages 142–150, Portland, Oregon, USA, June 2011. Association for Computational Linguistics

[Source: http://ai.stanford.edu/~amaas/data/sentiment/]

The dataset consists of 50,000 movie reviews from the original "train" and "test" subdirectories. The class labels are binary (1=positive and 0=negative) and contain 25,000 positive and 25,000 negative movie reviews, respectively.
For simplicity, I assembled the reviews in a single CSV file.


In [1]:
import pandas as pd
# if you want to download the original file:
#df = pd.read_csv('https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/50k_imdb_movie_reviews.csv')
# otherwise load local file
#df[['review', 'sentiment']].to_csv('shuffled_movie_data.csv', index=False)
df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,"This piece of crap, since I can't call it a mo...",0
49996,This indie film looks at the lives of a group ...,1
49997,Bride of Chucky starts late one night as Offic...,1
49998,Frank Sinatra starred in this odd little short...,1
49999,I saw this black and white comedy noir yesterd...,1


Let us shuffle the class labels.

In [2]:
import numpy as np
## uncomment these lines if you have dowloaded the original file:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df[['review', 'sentiment']].to_csv('shuffled_movie_data.csv', index=False)
df.head()

,review,sentiment
11841,This is the movie that finally pushed me over ...,1
19602,So what is one to do if you are a porno star w...,0
45519,Most of the French films I've seen - and enjoy...,0
25747,I am a big fan of Stephen King. I loved The Ru...,0
42642,"Yes Pigeon and Coburn are great, and it's inte...",0


<br>
<br>

## Preprocessing Text Data

Now, let us define a simple `tokenizer` that splits the text into individual word tokens. Furthermore, we will use some simple regular expression to remove HTML markup and all non-letter characters but "emoticons," convert the text to lower case, remove stopwords, and apply the Porter stemming algorithm to convert the words into their root form.

In [3]:
import numpy as np
# from nltk.stem.porter import PorterStemmer
import re
import sys
import time
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords

# stop = stopwords.words('english')
# porter = PorterStemmer()

# def tokenizer(text):
#     text = re.sub('<[^>]*>', '', text)
#     emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
#     text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
#     text = [w for w in text.split() if w not in stop]
#     tokenized = [porter.stem(w) for w in text]
#     return text

from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower())
    text = [w for w in text.split()]
    tokenized = [wordnet_lemmatizer.lemmatize(w) for w in text]
    return tokenized

Let's give it at try:

In [4]:
tokenizer('This :) is a <a> test! :-)</br>')

['this', 'is', 'a', 'test']

## Learning (SciKit)

First, we define a generator that returns the document body and the corresponding class label:

In [5]:
def stream_docs(path):
    with open(path, 'r', encoding="utf8") as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

To conform that the `stream_docs` function fetches the documents as intended, let us execute the following code snippet before we implement the `get_minibatch` function:

In [6]:
next(stream_docs(path='shuffled_movie_data.csv'))

('"This is the movie that finally pushed me over the line into registering with IMDb so that I could vote for (and comment on) it. I\'ve only recently come to appreciate well-produced ""war"" movies, and this is one of the most thoughtful I\'ve seen.<br /><br />""Stunning"" is the word that comes to mind when I think of this viewing experience. My husband and I watched this film last night for the first time. It is gently moving, yet exciting at the same time (not a contradiction). This story in the hands of Hollywood could have become just another smarmy, action-packed, Top Gun time-waster.<br /><br />The two lead actors playing Frantisek and Karel played off of each other marvelously well; and Krystof Hadek is a very ""pretty"" boy without seeming to exaggerate or exploit that fact. In terms of Hadek\'s acting ability and appearance, my husband said (tongue-in-cheek), ""Well, he\'s no Tom Cruise."" I replied, ""Thank God!"" If you appreciate beautiful and understated acting, see this

After we confirmed that our `stream_docs` functions works, we will now implement a `get_minibatch` function to fetch a specified number (`size`) of documents:

In [7]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return np.array(docs), np.array(y)

In [8]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

doc_generator = stream_docs(path='shuffled_movie_data.csv')

sentences = []
labels    = []
lengths   = []
for idx, review in enumerate(doc_generator):
    toVec = tokenizer(review[0])
    sentences.append(toVec)
    labels.append(review[1])
    lengths.append(len(toVec))
    sys.stdout.write('\r{:5.2f}%'.format(100*(idx+1)/50000))
sys.stdout.write('\rDone     \n\n')  
print(len(sentences))

D:\opt\windows\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Done     

50000


In [9]:
emb_size = 40

model = Word2Vec(sentences, size=emb_size, window=5, min_count=5, workers=4)
print('trained!')

trained!


In [10]:
sim = model.wv.most_similar(positive=['woman'], topn=1)
print("'woman' is similar to '{}' with a score of {:1.4f}".format(sim[0][0],sim[0][1]))

'woman' is similar to 'girl' with a score of 0.8719


In [11]:
print('vocabulary size :', len(model.wv.vocab.keys()))

vocabulary size : 35327


In [12]:
print('embedding size : ', model.wv['woman'].shape)

embedding size :  (40,)


In [13]:
class ReviewVectorizer:
    def __init__(self, model, maxlen):
        self.model  = model
        self.maxlen = maxlen
    def transform(self, reviews_tokenized):
        n = len(reviews_tokenized)
        vector = np.zeros((n, self.maxlen, emb_size), dtype=np.float16)
        for idx, review in enumerate(reviews_tokenized):
            for iw, word in enumerate(review):
                if word in self.model.wv.vocab:
                    vector[idx][iw] = self.model.wv[word]
            sys.stdout.write('\r{:5.2f}%'.format(100*(idx+1)/n))
        sys.stdout.write('\rDone     \n\n')                    
        vector = vector.reshape((n, -1))
        return vector

In [14]:
MAXLEN = max(lengths)
vectorizer = ReviewVectorizer(model, MAXLEN)

In [15]:
res = vectorizer.transform(sentences)

Done     



In [16]:
assert (MAXLEN*emb_size == res.shape[1])

In [17]:
y_train = np.asarray(labels).reshape((-1, 1))
X_train = res
del res
print('X_train shape: ',X_train.shape)
print('y_train shape: ',y_train.shape)

X_train shape:  (50000, 99920)
y_train shape:  (50000, 1)


Next, we will make use of the "hashing trick" through scikit-learns [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) to create a bag-of-words model of our documents. Details of the bag-of-words model for document classification can be found at  [Naive Bayes and Text Classification I - Introduction and Theory](http://arxiv.org/abs/1410.5329).

In [18]:
# from sklearn.feature_extraction.text import HashingVectorizer
# vect = HashingVectorizer(decode_error='ignore', 
#                          n_features=2**21,
#                          preprocessor=None, 
#                          tokenizer=tokenizer)

# Exercise 1: define features based on word embeddings (pre-trained word2vec / Glove/Fastext emebddings can be used)
# Define suitable d dimension, and sequence length
# https://nathanrooy.github.io/posts/2018-03-22/word2vec-from-scratch-with-python-and-numpy/


# doc_stream = stream_docs(path='shuffled_movie_data.csv')
# X_train, y_train = get_minibatch(doc_stream, size=50000)
# X_test, y_test = get_minibatch(doc_stream, size=5000)

In [19]:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 500) 
# train_data_features = vectorizer.fit_transform(X_train)
# X_train_vec = vectorizer.transform(X_train).toarray()

# X_train_vec.shape

Using the [SGDClassifier]() from scikit-learn, we will can instanciate a logistic regression classifier that learns from the documents incrementally using stochastic gradient descent. 

In [20]:
# from sklearn.linear_model import SGDClassifier
# clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
# doc_stream = stream_docs(path='shuffled_movie_data.csv')
# Exercise 2: Define at least a Three layer neural network. Define its structure (number of hidden neurons, etc)
# Define a nonlinear function for hidden layers.
# Define a suitable loss function for binary classification
# Implement the backpropagation algorithm for this structure
# Do not use Keras / Tensorflow /PyTorch etc. libraries
# Train the model using SGD


Red Neuronal de 3 capas:

In [21]:
D = X_train.shape[1]
K = 2

# Numero de neuronas por capas
nn_layer_0 = D
nn_layer_1 = 200
nn_layer_2 = 200
nn_layer_3 = K

def initialize_parameters(nn_input_dim, nn_hdim, nn_output_dim):
    # First layer weights
    np.random.seed(0)
    mu, sigma = 0, 0.1
    
    W1 = np.random.normal(mu, sigma, (nn_hdim, nn_input_dim))
#     W1 = np.zeros((nn_input_dim, nn_hdim_1))
    
    # First layer bias
#     b1 = np.random.normal(mu, sigma, (nn_hdim_1, 1))
    b1 = np.zeros((nn_hdim, 1))
    
    # Second layer weights
    W2 = np.random.normal(mu, sigma, (nn_hdim, nn_hdim))
#     W2 = np.zeros((nn_hdim_1, nn_hdim_2))
    
    # Second layer bias
#     b2 = np.random.normal(mu, sigma, (nn_hdim_2, 1))
    b2 = np.zeros((nn_hdim, 1))
    
    # Thirs layer weights
    W3 = np.random.normal(mu, sigma, (nn_output_dim, nn_hdim))
#     W3 = np.zeros((nn_hdim_2, nn_output_dim))
    
    # Second layer bias
#     b3 = np.random.normal(mu, sigma, (nn_output_dim, 1))
    b3 = np.zeros((nn_output_dim, 1))
    
    
    # Package and return model
    model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2, 'W3': W3, 'b3': b3}
    return model

Non Linear function for hidden layers:

In [22]:
def relu(x):
    return np.maximum(x, 0)

In [23]:
def relu_deriv(x):
    x[x<=0] = 0.0
    x[x>0] = 1.0
#     for i in range(x.shape[0]):
#         x[i] = [(el>0 and 1 or 0) for el in x[i,:]]
    return x

Non Linear function for output layers:

Softmax:

In [24]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    s = 1/(1.0+np.exp(-z))
    return s

In [25]:
def sigmoid_deriv(z):
    """
    Compute the derivative of the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid_deriv(z)
    """
    s = sigmoid(z)*(1-sigmoid(z))
    return s

loss function for binary classification:

In [26]:
def compute_cost(y, y_hat):
    m = y.shape[1]
    L = -1/m*np.sum(y*np.log(y_hat)+(1-y)*np.log(1-y_hat))
    return L

In [27]:
# def delta_cross_entropy(y, y_hat):    
#     ### START CODE HERE ### (≈ 4 lines of code)

#     delta = (y_hat-y)*sigmoid_deriv()
#     ### END CODE HERE ###
    
#     return y_hat

In [28]:
def propagate_two_layer(W1, b1, W2, b2, W3, b3, X, y, use_reg=False, reg_lambda=0.01):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    W -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dW -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[0]
#     print("propagate_two_layer X.shape: ", m)
    num_examples = X.shape[0]
    # FORWARD PROPAGATION (FROM X TO COST)
    ### START CODE HERE ### (≈ 5 lines of code), one hidden layer, first layer with RELU, activation: softmax

#     X12 = np.maximum(np.dot(W1.T, X.T)+b1, 0)
    A0 = X.T
#     A1 = np.tanh(np.dot(W1.T, A0)+b1)
    Z1 = np.dot(W1, A0)+b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1)+b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2)+b3
    A3 = sigmoid(Z3)
    
    cost = compute_cost(y.T, y_hat=A3)
    
    ### END CODE HERE ###
    
    if use_reg:
        ### START CODE HERE ###  (≈ 2 add regularization here)
        
        reg_cost = 0.5 * reg_lambda * np.sum(W1*W1) + 0.5 * reg_lambda * np.sum(W2*W2) + 0.5 * reg_lambda * np.sum(W3*W3)
        cost = cost + reg_cost
        
        ### END CODE HERE ###
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    ### START CODE HERE ### (≈ 3 lines of code)
    
#     dz3 = delta_cross_entropy(y=y, y_hat=A3)
    dz3 = (A3-y.T)*sigmoid_deriv(Z3)
  
    # backpropate the gradient to the parameters
    # first backprop into parameters W2 and b2
    dW3 = np.dot(dz3, A2.T)/m
    db3 = np.sum(dz3, axis=1)[:, np.newaxis]/m
    
    # next backprop into hidden layer
#     dhidden = ...
    dz2 = np.dot(W3.T, dz3)*relu_deriv(Z2)
    dW2 = np.dot(dz2, A1.T)/m
    db2 = np.sum(dz2, axis=1)[:, np.newaxis]/m
    ### END CODE HERE ###
        
    # next backprop into hidden layer
#     dhidden = ...
    dz1 = np.dot(W2.T, dz2)*relu_deriv(Z1)
    dW1 = np.dot(dz1, A0.T)/m
    db1 = np.sum(dz1, axis=1)[:, np.newaxis]/m
    ### END CODE HERE ###
    
    if use_reg:
        ### START CODE HERE ###  (≈ 2 add backprop regularization here)
        dW1 += reg_lambda*W1
        dW2 += reg_lambda*W2
        dW3 += reg_lambda*W3
        ### END CODE HERE ###

    assert(dW1.shape == W1.shape)
    assert(db1.dtype == float)
    assert(dW2.shape == W2.shape)
    assert(db2.dtype == float)
    assert(dW3.shape == W3.shape)
    assert(db3.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2,
             "dW3": dW3,
             "db3": db3}
    
    return grads, cost

In [29]:
def optimize_two_layer(W1, b1, W2, b2, W3, b3, X, y, num_iterations, learning_rate, use_reg = False, reg_lambda = 0.01, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    use_reg -- use regularization
    reg_lambda -- regularization weight
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    cost_i = 0
    for ii in range(num_iterations):
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        ### START CODE HERE ### 
        
        grads, cost = propagate_two_layer(W1, b1, W2, b2, W3, b3, X, y, use_reg=use_reg, reg_lambda=reg_lambda)

        ### END CODE HERE ###
        
        # Retrieve derivatives from grads
        dW1 = grads["dW1"]
        db1 = grads["db1"]
        dW2 = grads["dW2"]
        db2 = grads["db2"]
        dW3 = grads["dW3"]
        db3 = grads["db3"]
        
        # update rule (≈ 4 lines of code)
        ### START CODE HERE ###
        W1 = W1 - learning_rate*dW1
        b1 = b1 - learning_rate*db1
        W2 = W2 - learning_rate*dW2
        b2 = b2 - learning_rate*db2
        W3 = W3 - learning_rate*dW3
        b3 = b3 - learning_rate*db3
        ### END CODE HERE ###
        
        # Record the costs
        if ii % 100 == 0:
            costs.append(cost)
            cost_i += 1
        
        # Print the cost every 200 training iterations
        if print_cost and ii % 100 == 0:
            print ("Cost after iteration %i: %f" %(ii, cost))
            
#         if ((ii>100) and (np.abs(costs[cost_i-2]-costs[cost_i-1])<0.01)):
#           break
    
    params = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3}
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2,
             "dW3": dW3,
             "db3": db3}
    
    return params, grads, costs

In [33]:
def predict_two_layer(W1, b1, W2, b2, W3, b3, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    m = X.shape[0]
    Y_prediction = np.zeros((1,m))
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    ### START CODE HERE ### (≈ 3 lines of code)

    A0 = X.T
#     A1 = np.tanh(np.dot(W1.T, X.T)+b1)
    A1 = relu(np.dot(W1, A0)+b1)
    A2 = relu(np.dot(W2, A1)+b2)
    A3 = sigmoid(np.dot(W3, A2)+b3)

    ### END CODE HERE ###
    
    for ii in range(A3.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        ### START CODE HERE ### (≈ 4 lines of code)

        if(A3[0, ii] > 0.5):
            Y_prediction[0, ii] = 1
        else:
            Y_prediction[0, ii] = 0
        
        ### END CODE HERE ###
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [34]:
def train_two_layer(X_train, y_train, K=2, h_neurons=100, num_iterations=2000, learning_rate=0.5, use_reg=False, reg_lambda=0.01, init_type='random', print_cost=False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
    ### START CODE HERE ###
    
    # initialize parameters with zeros (≈ 2 lines of code)
    Y_prediction_train = 0
    costs_list = []
    model = initialize_parameters(D, h_neurons, K)
    W1, b1, W2, b2, W3, b3 = model['W1'], model['b1'], model['W2'], model['b2'], model['W3'], model['b3']

    # Gradient descent (≈ 1 line of code)
    for i in range(5):
        X_train_batch = X_train[(i*10000):((i+1)*10000),:]
        y_train_batch = y_train[(i*10000):((i+1)*10000)]

#         print("X_train_batch.shape: ", X_train_batch.shape)
        parameters, grads, costs = optimize_two_layer(W1, b1, W2, b2, W3, b3, X_train_batch, y_train_batch, num_iterations, learning_rate, use_reg = use_reg, reg_lambda = reg_lambda, print_cost = print_cost)
        costs_list.append(costs)
        # Retrieve parameters w and b from dictionary "parameters"
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        W3 = parameters["W3"]
        b3 = parameters["b3"]

        Y_prediction_train = predict_two_layer(W1, b1, W2, b2, W3, b3, X_train_batch)

        # Print train/test Errors
        print('training accuracy: %.2f' % (np.mean(Y_prediction_train == y_train_batch.T)))
        print('Y_prediction_train.shape: ', Y_prediction_train.shape)
    
    d = {"costs": costs_list,
         "W1" : W1, 
         "b1" : b1,
         "W2" : W2, 
         "b2" : b2,
         "W3" : W3, 
         "b3" : b3,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations,
         "predictions": Y_prediction_train}
    
    return d

In [ ]:
%xmode Plain
%pdb on
d = train_two_layer(X_train, y_train, K=1,  h_neurons=20, num_iterations=500, learning_rate= 1, use_reg=True, reg_lambda=1e-2, print_cost=True)

Exception reporting mode: Plain
Automatic pdb calling has been turned ON
Cost after iteration 0: 100.681798
Cost after iteration 100: 13.478556
Cost after iteration 200: 2.147620
Cost after iteration 300: 0.661567
Cost after iteration 400: 5.502520
training accuracy: 0.58
Y_prediction_train.shape:  (1, 1000)
Cost after iteration 0: 0.736600
Cost after iteration 100: 0.653064
Cost after iteration 200: 0.345408
Cost after iteration 300: 0.532044
Cost after iteration 400: 0.061171
training accuracy: 0.78
Y_prediction_train.shape:  (1, 1000)
Cost after iteration 0: 0.697742
Cost after iteration 100: 0.698159
training accuracy: 0.76
Y_prediction_train.shape:  (1, 1000)
Cost after iteration 0: 0.683611
Cost after iteration 100: 0.697352
Cost after iteration 200: 0.723115
Cost after iteration 300: 0.707135
Cost after iteration 400: 0.722753
training accuracy: 0.68
Y_prediction_train.shape:  (1, 1000)
Cost after iteration 0: 0.703770
Cost after iteration 100: 0.674737
Cost after iteration 200:

In [ ]:
#import pyprind
#pbar = pyprind.ProgBar(45)

# classes = np.array([0, 1])
# for _ in range(45):
#     X_train, y_train = get_minibatch(doc_stream, size=1000)
#     X_train = vect.transform(X_train).toarray()
#     clf.partial_fit(X_train, y_train, classes=classes)
#     #pbar.update()
    
Y_prediction_train = predict_two_layer(d["W1"], d["b1"], d["W2"], d["b2"], d["W3"], d["b3"], X_train[0:10000,:])
print('training accuracy: %.2f' % (np.mean(Y_prediction_train == y_train[0:10000].T)))

Depending on your machine, it will take about 2-3 minutes to stream the documents and learn the weights for the logistic regression model to classify "new" movie reviews. Executing the preceding code, we used the first 45,000 movie reviews to train the classifier, which means that we have 5,000 reviews left for testing:

In [ ]:
# X_test, y_test = get_minibatch(doc_stream, size=5000)
# X_test = vect.transform(X_test)
# print('Accuracy: %.3f' % clf.score(X_test, y_test))
#Exercise 3: compare  with your Neural Network

I think that the predictive performance, an accuracy of ~87%, is quite "reasonable" given that we "only" used the default parameters and didn't do any hyperparameter optimization. 

After we estimated the model perfomance, let us use those last 5,000 test samples to update our model.

In [ ]:
clf = clf.partial_fit(X_test, y_test)

<br>
<br>

# Model Persistence

In the previous section, we successfully trained a model to predict the sentiment of a movie review. Unfortunately, if we'd close this IPython notebook at this point, we'd have to go through the whole learning process again and again if we'd want to make a prediction on "new data."

So, to reuse this model, we could use the [`pickle`](https://docs.python.org/3.5/library/pickle.html) module to "serialize a Python object structure". Or even better, we could use the [`joblib`](https://pypi.python.org/pypi/joblib) library, which handles large NumPy arrays more efficiently.

To install:
conda install -c anaconda joblib

In [ ]:
import joblib
import os
if not os.path.exists('./pkl_objects'):
    os.mkdir('./pkl_objects')
    
joblib.dump(vect, './vectorizer.pkl')
joblib.dump(clf, './clf.pkl')

Using the code above, we "pickled" the `HashingVectorizer` and the `SGDClassifier` so that we can re-use those objects later. However, `pickle` and `joblib` have a known issue with `pickling` objects or functions from a `__main__` block and we'd get an `AttributeError: Can't get attribute [x] on <module '__main__'>` if we'd unpickle it later. Thus, to pickle the `tokenizer` function, we can write it to a file and import it to get the `namespace` "right".

In [ ]:
%%writefile tokenizer.py
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

In [ ]:
from tokenizer import tokenizer
joblib.dump(tokenizer, './tokenizer.pkl')

Now, let us restart this IPython notebook and check if the we can load our serialized objects:

In [ ]:
import joblib
tokenizer = joblib.load('./tokenizer.pkl')
vect = joblib.load('./vectorizer.pkl')
clf = joblib.load('./clf.pkl')

After loading the `tokenizer`, `HashingVectorizer`, and the tranined logistic regression model, we can use it to make predictions on new data, which can be useful, for example, if we'd want to embed our classifier into a web application -- a topic for another IPython notebook.

In [ ]:
example = ['I did not like this movie']
X = vect.transform(example)
clf.predict(X)

In [ ]:
example = ['I loved this movie']
X = vect.transform(example)
clf.predict(X)